In [16]:
import os
import os.path
import sys
import glob
import numpy as np
import scipy.optimize    as op
import matplotlib.pyplot as plt
import urllib.request
from astropy.io import fits 
from astropy.table import Table
import pandas as pd

def redshift(loc):
    z = fits.open(loc)
    val = z[2].data
    z.close()
    return val['Z'][0]

local_dir="/home/colgoat/Jupyter/Entorno/Spec"
file="Finalsubcat_DR14.txt"
local_file = os.path.join(local_dir,file)
df = pd.read_csv(local_file,delim_whitespace=True, skiprows=0,dtype = {'id':str})
catlist = df['id']

catlist

0         3586/spec-3586-55181-0158.fits
1         3586/spec-3586-55181-0188.fits
2         3586/spec-3586-55181-0228.fits
3         3586/spec-3586-55181-0312.fits
4         3586/spec-3586-55181-0320.fits
                       ...              
168040    7517/spec-7517-56772-0020.fits
168041    7517/spec-7517-56772-0024.fits
168042    7517/spec-7517-56772-0038.fits
168043    7517/spec-7517-56772-0047.fits
168044    7517/spec-7517-56772-0053.fits
Name: id, Length: 168045, dtype: object

In [17]:
direc = "/home/colgoat/Jupyter/Entorno/Spec/"
si_list = []
no_list = []

for a in range(len(catlist)):
    if (os.path.isfile(direc+catlist[a]) == True):
        si_list.append(direc+catlist[a])
    else:
        no_list.append(direc+catlist[a])

np.savetxt('/home/colgoat/Jupyter/Entorno/Spec/found.txt',si_list, fmt='%s', delimiter=',')
np.savetxt('/home/colgoat/Jupyter/Entorno/Spec/not_found.txt',no_list, fmt='%s', delimiter=',')

len(si_list)

11

In [28]:
limits = [1050, 1093.3, 1136.6, 1180] # (Three forest subdivisions) & The Lyman-a line
lya = 1215.67
snr = [0,0,0]
z_for = [0,0,0]
SNR_F = []

for a in range(11):
    file = fits.open(si_list[a])
    spectra = file[1].data
    for b in range(3):
        wave_cut=((10**spectra['loglam'])>(1. + redshift(si_list[a])) * limits[b]) \
        & ((10**spectra['loglam'])<(1. + redshift(si_list[a])) * limits[b+1])
        
        forest = spectra[wave_cut]
        
        raw_snr = forest['flux'] * np.sqrt(forest['ivar'])
        
        snr[b] = np.mean(raw_snr)
        
        raw_z = ((10**forest['loglam'])/lya)-1
        
        z_for[b] = np.mean(raw_z)
        
    file.close()   
    SNR_F.append("ID: " + si_list[a][40:60]+" Z_QSO: "+str(redshift(si_list[a]))+" SubforestSNR: " \
    +str(snr[0])+" "+str(snr[1])+" "+str(snr[2])+" z_forest: "+str(z_for[0])+" "+str(z_for[1])+" "+str(z_for[2]))
    
    snr = [0,0,0]
    z_for = [0,0,0]
    
SNR_F



/home/colgoat/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/colgoat/.local/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


['ID: spec-4289-55856-0532 Z_QSO: 2.39217 SubforestSNR: 0.90047354 1.341343 1.8761525 z_forest: 2.0019495 2.1107233 2.2315419',
 'ID: spec-4629-55630-0536 Z_QSO: 3.0992098 SubforestSNR: 2.986251 3.203494 3.3270519 z_forest: 2.612967 2.7593408 2.9053457',
 'ID: spec-5006-55706-0452 Z_QSO: 2.3611636 SubforestSNR: 0.9722188 1.2114176 1.9686958 z_forest: 1.9740868 2.0822039 2.2019148',
 'ID: spec-5448-56013-0144 Z_QSO: 2.305159 SubforestSNR: 1.9284124 2.9875946 4.6245346 z_forest: 1.9514673 2.0308285 2.1489043',
 'ID: spec-5801-56328-0944 Z_QSO: 3.1290314 SubforestSNR: 2.843162 2.3872023 2.405431 z_forest: 2.6392646 2.7862709 2.9337773',
 'ID: spec-6138-56598-0467 Z_QSO: 2.2299812 SubforestSNR: nan 3.719985 6.631879 z_forest: nan 1.9754353 2.0772264',
 'ID: spec-6182-56190-0718 Z_QSO: 2.9955041 SubforestSNR: 5.6637664 5.7456937 4.000906 z_forest: 2.5217943 2.6640499 2.8063595',
 'ID: spec-6253-56574-0924 Z_QSO: 2.3892016 SubforestSNR: 0.5887586 0.8315658 1.779615 z_forest: 2.0081327 2.1082